In [3]:
from dotenv import load_dotenv
import requests
import pandas as pd
import time
import os

In [5]:
load_dotenv()

API_KEY = os.getenv("API_KEY_RAWG")
print(API_KEY)

654f26e5296c40d69fafe34f6d5ac5d9


In [8]:
def fetch_rawg_games(api_key=None, max_pages=500, page_size=40):
    """
    Récupère les jeux depuis RAWG API, paginés.
    Args:
        api_key (str): La clé API RAWG 
        max_pages (int): nombre de pages à récupérer (chaque page contient page_size jeux)
        page_size (int): nombre de jeux par page (max 40 recommandé)
    
    Returns:
        list: liste brute des jeux (dictionnaires JSON)
    """
    all_games = []
    base_url = "https://api.rawg.io/api/games"
    
    for page in range(1, max_pages + 1):
        params = {
            "page": page,
            "page_size": page_size,
            "ordering": "-added",  # trie par popularité
        }
        if api_key:
            params["key"] = api_key
        
        print(f"Récupération page {page}...")
        response = requests.get(base_url, params=params)
        if response.status_code != 200:
            print(f"Erreur API page {page}: {response.status_code}")
            break
        
        data = response.json()
        all_games.extend(data.get("results", []))
        
        # Pause pour éviter de spammer l'API
        time.sleep(1)
        
    print(f"Total jeux récupérés : {len(all_games)}")
    
    return all_games

def clean_games_data(games_list):
    """
    Nettoie et structure la liste brute des jeux en DataFrame pandas.
    Args:
        games_list (list): liste de jeux (dictionnaires)
    
    Returns:
        pd.DataFrame: dataframe nettoyé avec colonnes sélectionnées et formats adaptés
    """
    df = pd.DataFrame(games_list)
    
    # Colonnes clés à garder
    cols_to_keep = ["id", "name", "slug","background_image","released", "rating", "ratings_count", "metacritic", "genres", "platforms","stores","esrb_rating"]
    df = df[cols_to_keep]
    
    # Extraire noms des genres (liste de dicts -> liste de noms)
    df["genres"] = df["genres"].apply(lambda gs: [g["name"] for g in gs] if isinstance(gs, list) else [])
    
    # Extraire noms des plateformes (liste complexes)
    def extract_platforms(plats):
        if isinstance(plats, list):
            return [p["platform"]["name"] for p in plats if "platform" in p]
        return []
    df["platforms"] = df["platforms"].apply(extract_platforms)
    
    # Extraire noms des stores (liste complexes)
    def extract_stores(stores):
        if isinstance(stores, list):
            return [s["store"]["name"] for s in stores if "store" in s]
        return []
    df["stores"] = df["stores"].apply(extract_stores)
  
    # Extraire tag ESRB
    df["esrb_rating"] = df["esrb_rating"].apply(lambda x: x["name"] if isinstance(x, dict) else "")

    # Convertir date en datetime (et gérer valeurs manquantes)
    df["released"] = pd.to_datetime(df["released"], errors='coerce')
    
    # Remplacer NaN rating par 0
    df["rating"] = df["rating"].fillna(0)
    df["ratings_count"] = df["ratings_count"].fillna(0).astype(int)
    df["metacritic"] = df["metacritic"].fillna(0)
    
    return df

def main():
    # Étape 1 : collecte des données RAWG (500 pages x 40 jeux = max 20 000 jeux)
    raw_games = fetch_rawg_games(api_key=API_KEY, max_pages=500, page_size=40)
    
    # Étape 2 : nettoyage et structuration
    df_games = clean_games_data(raw_games)
    print (df_games)
    # Étape 3 : sauvegarde en CSV
    output_file = "C:/Users/sirnb/OneDrive/Bureau/WildCode/Projets/Projet_3/data-game/csv/jeux_video_rawg.csv"
    df_games.to_csv(output_file, index=False, encoding='utf-8-sig')
    print(f"Données nettoyées sauvegardées dans : {output_file}")

if __name__ == "__main__":
    main()


Récupération page 1...
Récupération page 2...
Récupération page 3...
Récupération page 4...
Récupération page 5...
Récupération page 6...
Récupération page 7...
Récupération page 8...
Récupération page 9...
Récupération page 10...
Récupération page 11...
Récupération page 12...
Récupération page 13...
Récupération page 14...
Récupération page 15...
Récupération page 16...
Récupération page 17...
Récupération page 18...
Récupération page 19...
Récupération page 20...
Récupération page 21...
Récupération page 22...
Récupération page 23...
Récupération page 24...
Récupération page 25...
Récupération page 26...
Récupération page 27...
Récupération page 28...
Récupération page 29...
Récupération page 30...
Récupération page 31...
Récupération page 32...
Récupération page 33...
Récupération page 34...
Récupération page 35...
Récupération page 36...
Récupération page 37...
Récupération page 38...
Récupération page 39...
Récupération page 40...
Récupération page 41...
Récupération page 42...
R

In [12]:
jeux = pd.read_csv(r"C:\Users\sirnb\OneDrive\Bureau\WildCode\Projets\Projet_3\data-game\csv\jeux_video_rawg.csv")

jeux

,id,name,slug,background_image,released,rating,ratings_count,metacritic,genres,platforms,stores,esrb_rating
0,3498,Grand Theft Auto V,grand-theft-auto-v,https://media.rawg.io/media/games/20a/20aa03a1...,2013-09-17,4.47,7128,92.0,['Action'],"['PC', 'PlayStation 5', 'Xbox Series S/X', 'Pl...","['Steam', 'PlayStation Store', 'Epic Games', '...",Mature
1,3328,The Witcher 3: Wild Hunt,the-witcher-3-wild-hunt,https://media.rawg.io/media/games/618/618c2031...,2015-05-18,4.64,6925,92.0,"['Action', 'RPG']","['Xbox Series S/X', 'PlayStation 5', 'macOS', ...","['GOG', 'PlayStation Store', 'Steam', 'Xbox St...",Mature
2,4200,Portal 2,portal-2,https://media.rawg.io/media/games/2ba/2bac0e87...,2011-04-18,4.59,5916,95.0,"['Shooter', 'Puzzle']","['PlayStation 3', 'PC', 'Xbox 360', 'Linux', '...","['Xbox Store', 'Steam', 'PlayStation Store', '...",Everyone 10+
3,4291,Counter-Strike: Global Offensive,counter-strike-global-offensive,https://media.rawg.io/media/games/736/73619bd3...,2012-08-21,3.57,3581,81.0,['Shooter'],"['PC', 'Linux', 'Xbox 360', 'PlayStation 3']","['PlayStation Store', 'Steam', 'Xbox 360 Store']",Mature
4,5286,Tomb Raider (2013),tomb-raider,https://media.rawg.io/media/games/021/021c4e21...,2013-03-05,4.06,4009,86.0,['Action'],"['PlayStation 3', 'Xbox 360', 'macOS', 'PC', '...","['Xbox 360 Store', 'Steam', 'PlayStation Store...",Mature
...,...,...,...,...,...,...,...,...,...,...,...,...
16835,16650,123 Slaughter Me Street 2,123-slaughter-me-street-2,https://media.rawg.io/media/screenshots/323/32...,2016-10-31,0.00,4,0.0,"['Action', 'Adventure', 'Strategy']",['PC'],['Steam'],NaN
16836,16363,Sleight,sleight,https://media.rawg.io/media/screenshots/bf8/bf...,2016-11-09,0.00,3,0.0,"['Action', 'Indie']",['PC'],['Steam'],NaN
16837,16333,Kyoto Colorful Days,kyoto-colorful-days,https://media.rawg.io/media/screenshots/a9f/a9...,2016-06-14,0.00,3,0.0,['Adventure'],['PC'],['Steam'],NaN
16838,15823,Ghostship Aftermath,ghostship-aftermath,https://media.rawg.io/media/screenshots/f64/f6...,2014-07-18,0.00,1,0.0,"['Action', 'Adventure', 'Indie']",['PC'],['Steam'],NaN
